In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
#from official.nlp import optimization  # to create AdamW optimizer
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv('data.csv', encoding='utf-8', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text'])
data.drop(columns=['ids', 'date', 'flag', 'user'], inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data = shuffle(data, random_state=42).reset_index(drop=True)
data

In [ ]:
df = data

In [ ]:
# Convert 'target' to bnary sentiment labels (0 or 1)
df['target'] = df['target'].apply(lambda x: 0 if x == 0 else 1)

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert data to TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices((train_df['text'].values, train_df['target'].values))
test_data = tf.data.Dataset.from_tensor_slices((test_df['text'].values, test_df['target'].values))


In [ ]:
bert_model_url = "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2"  # SST-2 model trained for sentiment analysis
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

bert_preprocessor = hub.KerasLayer(preprocessor_url)
bert_encoder = hub.KerasLayer(bert_model_url)

In [ ]:
def build_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocessor(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Extract the pooled output from the BERT encoder
    net = outputs['pooled_output']
    
    # Add dropout for regularization
    net = tf.keras.layers.Dropout(0.1)(net)
    
    # Add additional dense layers with ReLU activation
    net = tf.keras.layers.Dense(128, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)  # Add dropout to the new dense layer
    net = tf.keras.layers.Dense(128, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(64, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.4)(net)
    
    # Final output layer with sigmoid activation for binary classification
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)
    
    return tf.keras.Model(inputs=[text_input], outputs=[net])

# Initialize and build the model
model = build_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [ ]:
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
BATCH_SIZE = 32
train_data = train_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

with tf.device('/GPU:0'):
    model.fit(train_data, epochs=5)


Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'Equal_2' defined at (most recent call last):
    File "c:\Users\admin-npatel237\AppData\Local\anaconda3\envs\NLP\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\admin-npatel237\AppData\Local\anaconda3\envs\NLP\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\admin-npatel237\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\admin-npatel237\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\admin-npatel237\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\admin-npatel237\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", lin

In [ ]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy:.2f}")
    

In [ ]:
model.save('Best_82', include_optimizer=False)